In [82]:
from container import Container
from template import Template, Example
api_key = 'OpenAI-Key'
model = Container(api_key, 'text-davinci-003')

In [83]:
import pandas as pd
from dataload import parse_data
train = pd.read_json('train.json')
test = pd.read_json('example.json')
one_shot_train = train.loc[train['pairs'].apply(len) == 1].sample(1)
five_shot_train = train.loc[train['pairs'].apply(len) == 1].sample(3).append(
    train.loc[train['pairs'].apply(len) == 2].sample(1)).append(
    train.loc[train['pairs'].apply(len) == 3].sample(1))

/var/folders/sb/nbhxgzzj2ds2h3yjxjyn8f2w0000gn/T/ipykernel_11413/1599189418.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_shot_train = train.loc[train['pairs'].apply(len) == 1].sample(3).append(
/var/folders/sb/nbhxgzzj2ds2h3yjxjyn8f2w0000gn/T/ipykernel_11413/1599189418.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_shot_train = train.loc[train['pairs'].apply(len) == 1].sample(3).append(


In [84]:
parse_train_1 = parse_data(one_shot_train)
parse_train_5 = parse_data(five_shot_train)
# parse_train_6 = parse_data(ten_shot_train)
parse_test = parse_data(test)

In [85]:
# question_temp = '''Extract emotion-cause pairs and return a list of tuple based on the given content.
# In each tuple, the first element is the index of the emotion clause and the second element is the index of cause clause.'''

# question_temp_pair_num_limit = '''Extract 1 to 4 emotion-cause pairs and return a list of tuple based on the given content.
# In each tuple, the first element is the index of the emotion clause and the second element is the index of cause clause.'''

# question_temp_sequence_prompt = '''Extract emotion-cause pairs and return a list of tuple based on the given content.
# In each tuple, the first element is the index of the emotion clause and the second element is the index of cause clause.
# The cause clause usually appears in the front of emotion clause'''

# question_temp_both_hint = '''Extract 1 to 4 emotion-cause pairs and return a list of tuple based on the given content.
# In each tuple, the first element is the index of the emotion clause and the second element is the index of cause clause.
# The cause clause usually appears in the front of emotion clause'''

# DATA = parse_train_5
FILENAME = 'prompt_5_shot.txt'
RESULTNAME = 'result_5_shot.txt'
TASK_SIMBOL = '任务4'
TASK_SIMBOL2 = '：'
TASK_SIMBOL = 'Task 4'
TASK_SIMBOL2 = ':'
# QUESTION = question_temp

In [86]:
# few_shot = [{'content': e['content'], 'pair': e['pair']} for e in DATA]
dev = [{'content': e['content'], 'pair': e['pair']} for e in parse_test]

In [87]:
instruction = "文档：\""
question = '''\"
以上文档的每一行代表一个子句，每行开头的数字表示该子句的编号。
我们将包含情绪表达的子句称为“情绪子句”，将导致情绪发生的子句
称为“原因子句”。基于以上内容, 分别完成以下任务。1. 用一句话描
述以上文档中包含的情绪以及它对应的原因。2. 输出任务1中情绪子句
的编号, 你只需要找到情绪强度最强的那一个子句。3.对于任务2中的每
个情绪子句，找到导致其情绪的原因子句，并输出原因子句的编号，你
只需要找到最符合要求的那一个子句。4.将以上具有因果关系的子句匹
配成'(情绪子句编号，原因子句编号)'的形式，最后将所有的编号配对按
照集合的形式输出，例如 任务：[[1,2],[3,4]]。注意：情绪子句和原因子句有可
能是同一个子句，且最终只需要输出情绪和原因最明显的配对。'''

instruction = "content: \""
question = '''\"
Each line of the content represents a clause, and the number at the beginning of each line indicates the clause number. We refer to the clauses that contain emotional expressions as "emotion clauses" and the clauses that cause emotions as "cause clauses". Based on the content, complete the following tasks:
1. Describe the emotion contained in the document in one sentence and its corresponding cause.
2. Output the clause number of the emotion clause for Task 1. You only need to find the clause with the strongest emotional intensity.
3. For each emotion clause in Task 2, find the cause clause that caused the emotion and output the clause number of the cause clause. You only need to find the clause that best meets the requirements.
4. Match the clauses with causal relationships in the form of '[emotion clause number, cause clause number]', and output all paired numbers in list format, for example, Task 4: [[1,2],[3,4]]. 
Note: the emotion clause and the cause clause may be the same clause, and only the most obvious emotion-cause pairing needs to be outputted.
Besure to output in this format: Task 1: ... Task 2: ...'''

template = Template(instruction=instruction)
template(question=question)
# template(question=question, content=content)

['"\nEach line of the content represents a clause, and the number at the beginning of each line indicates the clause number. We refer to the clauses that contain emotional expressions as "emotion clauses" and the clauses that cause emotions as "cause clauses". Based on the content, complete the following tasks:\n1. Describe the emotion contained in the document in one sentence and its corresponding cause.\n2. Output the clause number of the emotion clause for Task 1. You only need to find the clause with the strongest emotional intensity.\n3. For each emotion clause in Task 2, find the cause clause that caused the emotion and output the clause number of the cause clause. You only need to find the clause that best meets the requirements.\n4. Match the clauses with causal relationships in the form of \'[emotion clause number, cause clause number]\', and output all paired numbers in list format, for example, Task 4: [[1,2],[3,4]]. \nNote: the emotion clause and the cause clause may be the

In [88]:
def few_shot_ecpe(model, template, content, temp=0.7, n=2):
    # train = few_shot[:10]
    example = Example(content=content)
    completion, prompt = model.chat(template, example, temp, n)
    return completion, prompt

In [89]:
from tqdm import tqdm
prompts = []
predict = []
result = []
for i in tqdm(range(100)):
    completion, prompt = few_shot_ecpe(model, template, dev[i]['content'], temp=0.3, n=1)
    # predict.append(completion.choices[0].text.split(':')[1])
    predict.append(completion.choices[0].text)
    prompts.append(prompt)
    result.append(dev[i]['pair'])

100%|██████████| 100/100 [07:57<00:00,  4.78s/it]


In [91]:
def parse_to_list(data):
    result = []
    for i in range(len(data)):
        index = data[i].find(TASK_SIMBOL)
        # print(data[i])
        predicted = data[i][index:]
        predicted = predicted.split(TASK_SIMBOL2)[1]
        predicted = predicted.replace('(', '[').replace(')', ']') #replace all () to []
        predicted = predicted.replace('。', '')
        predicted = predicted.strip() # strip all begining space
        predicted = eval(predicted)
        result.append(predicted)
    return result

predict = parse_to_list(predict)

In [92]:
def compare(Y1, Y2):
    '''Y1: Y_hat case, Y2: Y case'''
    correct = 0
    for c in Y2:
        if c in Y1:
            correct += 1
    return correct, len(Y1), len(Y2)

def calc_precision(Y_hat, Y):
    assert len(Y_hat) == len(Y), print('shape mismatched')
    correct = 0
    total = 0
    ground_true = 0
    for i in range(len(Y_hat)):
        c, t, a = compare(Y_hat[i], Y[i])
        correct += c
        total += t
        ground_true += a
    precision = correct/total
    recall = correct/ground_true
    f1 = 2 * precision * recall / (precision + recall)
    return precision, recall, f1, correct, total, ground_true

calc_precision(predict, result)


(0.11564625850340136, 0.14782608695652175, 0.12977099236641224, 17, 147, 115)